In [235]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [236]:
df = pd.read_csv('./Data.csv')
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [237]:
models = {
    'Logistic Regression': LogisticRegression(),
    'K-NN': KNeighborsClassifier,
    'SVC (Linear)': SVC(kernel = 'linear'), 
    'SVC (rbf)': SVC(kernel = 'rbf'), 
    'SVC (poly)': SVC(kernel = 'poly'),
    'SVC (sigmoid)': SVC(kernel = 'sigmoid'),
    'GauusianNB': GaussianNB(),
    'Random Forest': RandomForestClassifier(),
    'Decision Tree': DecisionTreeClassifier()
} 

In [238]:
sc = StandardScaler()

In [239]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)
X_scaled = sc.transform(X)

In [240]:
results = pd.DataFrame(columns = pd.MultiIndex.from_tuples(
(('TTS', 'Normal'), ('TTS', 'Scaled'), ('K-Fold', 'Normal'), ('K-Fold', 'Scaled'),)))

In [241]:
def TTS():
    for modelName in models:
        
        if modelName == 'K-NN':
            Scaled = 0
            
            for a in range(1, 50):
                model = models[modelName](a)
                model.fit(X_train_scaled, y_train)
                score = model.score(X_test_scaled, y_test)
                
                if score > Scaled:
                    Scaled = score
                    Normal = model.fit(X_train, y_train).score(X_test, y_test)
                    
        else:
            Normal = models[modelName].fit(X_train, y_train).score(X_test, y_test)
            Scaled = models[modelName].fit(X_train_scaled, y_train).score(X_test_scaled, y_test)
        
        results.loc[modelName] = {
            ('TTS', 'Normal'): f'{round(Normal * 100, 2)}%',
            ('TTS', 'Scaled'): f'{round(Scaled * 100, 2)}%'
        }

In [242]:
def kFold():
    for modelName in enumerate(models):

        if modelName[1] == 'K-NN':
            Scaled = 0
            
            for a in range(1, 50):
                
                if cross_val_score(models[modelName[1]](a), X, y).mean() > Scaled:
                    Scaled = cross_val_score(models[modelName[1]](a), X_scaled, y).mean()
                    Normal = cross_val_score(models[modelName[1]](a), X, y).mean()
        
        else:
            Normal = cross_val_score(models[modelName[1]], X, y, cv = 10).mean()
            Scaled = cross_val_score(models[modelName[1]], X_scaled, y, cv = 10).mean()
        
        results.iloc[modelName[0], 2] = f'{round(Normal * 100, 2)}%'
        results.iloc[modelName[0], 3] = f'{round(Scaled * 100, 2)}%'

In [243]:
TTS()
kFold()

In [ ]:
results

TTS          K-Fold        
                     Normal  Scaled  Normal  Scaled
Logistic Regression  60.23%  95.32%  65.01%  96.64%
K-NN                 59.65%  96.49%  53.16%  95.03%
SVC (rbf)            60.23%  95.32%  63.68%  96.64%
SVC (poly)           60.23%  91.81%  65.01%  95.33%
SVC (sigmoid)        60.82%  96.49%  56.62%  96.05%
GauusianNB           82.46%  96.49%  85.46%   96.2%
Random Forest        95.91%  95.91%  96.78%  96.49%
Decision Tree        94.74%  93.57%  94.58%  94.14%